In [ ]:

import bpy
import csv

# Configuration
csv_path = r'C:\Users\91910\Downloads\telemetry_data.csv' # Replace with your CSV path
fps = 24  # Frames per second

curve_name = "RacePath"  # Name of the curve
cube_name = "Cube"    # Name of the cube

# Read CSV data
with open(csv_path, newline='') as csvfile:
    reader = list(csv.DictReader(csvfile))
    times = [float(row['Time']) for row in reader]
    speeds = [float(row['Speed']) for row in reader]

# Normalize speeds (scaling them between 0 and 1)
max_speed = max(speeds)
min_speed = min(speeds)
normalized_speeds = [(speed - min_speed) / (max_speed - min_speed) for speed in speeds]

# Total animation frames based on the last time value in the CSV
total_time = times[-1]
total_frames = int(total_time * fps)

# Get the curve and cube objects
curve = bpy.data.objects.get(curve_name)
cube = bpy.data.objects.get(cube_name)

if not curve or not cube:
    raise ValueError(f"Curve '{curve_name}' or Cube '{cube_name}' not found in the scene.")

# Ensure path animation is enabled for the curve
curve.data.use_path = True
curve.data.path_duration = total_frames  # Set the path duration to match the animation

# Add Follow Path constraint to the cube (if not already added)
constraint = cube.constraints.get("Follow Path")
if not constraint:
    constraint = cube.constraints.new(type='FOLLOW_PATH')
    constraint.target = curve
    constraint.use_curve_follow = True

# Remove any existing animation on offset_factor
if "offset_factor" in [fc.data_path for fc in cube.animation_data.action.fcurves]:
    cube.animation_data_clear()

# Add keyframes for offset_factor to control speed
for i, time in enumerate(times):
    frame = int(time * fps)
    offset = normalized_speeds[i]
    constraint.offset_factor = offset
    constraint.keyframe_insert(data_path="offset_factor", frame=frame)

# Ensure interpolation is linear for precise speed changes
action = cube.animation_data.action
for fcurve in action.fcurves:
    if fcurve.data_path == "constraints[\"Follow Path\"].offset_factor":
        for keyframe in fcurve.keyframe_points:
            keyframe.interpolation = 'LINEAR'

# Set the frame range for the animation
bpy.context.scene.frame_start = 1
bpy.context.scene.frame_end = total_frames

print(f"Animation completed: {total_frames} frames with variable speed applied.")
